In [1]:
import os
import csv
from paddleocr import PaddleOCR, draw_ocr
from paddleocr import PPStructure, save_structure_res
import preprocess_report as pre

# Path to your CSV file
csv_file_path = 'datasets/metadata for reports/training_meta_data - Blad1.csv'
ocr = PaddleOCR(use_angle_cls=True, lang='sv', use_gpu=True)
table_engine = PPStructure(layout=True, show_log=True, lang='en', ocr = False,use_gpu=True)  #consider layout, less found tbales but better strucutre?

# Open the CSV file
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    # Create a CSV reader
    csv_reader = csv.DictReader(file)

    # Iterate over each row in the CSV
    testindex = 0
    for row in csv_reader:
        if testindex == 10:
            break
        testindex += 1
        # Extract the data you're interested in
        filename = row['FileName']
        approved = row['approved']
        result = row['resultat']
        balans = row['balans']

        if approved == 'Y':
            print(f'Filename: {filename}, resultat: {result}, balans: {balans}')

            result_pages = [] if result == "NA" else result.split('/')
            balans_pages = [] if balans == "NA" else balans.split('/')

            images = pre.convert_report_to_image(f"datasets/reports/{filename}")
            images = pre.crop_ndarrays_left_top_percentage(images)
            save_path = os.path.join('./images', filename)
            os.makedirs(save_path, exist_ok=True)
            resultImages = []
            resultImage = []
            for i in result_pages:
                resultImages.append(images[int(i)])
            if resultImages:

                imageIndex = 0
                for i in resultImages:
                    save_folder = f'./output/{filename}/'
                

                    file_path = os.path.join(save_path, f"resultat{imageIndex}.jpg")
                    #cv2.imwrite(file_path, i)

                    table = table_engine(i)
                    save_structure_res(table, save_folder, "result")
                    imageIndex += 1

            balansImages = []
            balansImage = []
            for i in balans_pages:
                balansImages.append(images[int(i)])

            if balansImages:
                imageIndex = 0
                for i in balansImages:
                    save_folder = f'./output/{filename}/'
                    img = balansImage
                    file_path = os.path.join(save_path, f"balans{imageIndex}.jpg")
                    #cv2.imwrite(file_path, i)
                    table = table_engine(i)
                    save_structure_res(table, save_folder, "balans")
                    imageIndex += 1



[2024/04/16 15:40:01] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e

RuntimeError: (PreconditionNotMet) Cannot load cudnn shared library. Cannot invoke method cudnnGetVersion.
  [Hint: cudnn_dso_handle should not be null.] (at /paddle/paddle/phi/backends/dynload/cudnn.cc:64)
